## Import module and start Aspen

In [ ]:
import aspen_tools
import analysis
import shutil

project_name = "test"

shutil.copy2("C:\\Users\\user\\Documents\\Carbonfix_package\\Template.apwz", f"C:\\Users\\user\\Documents\\Carbonfix_package\\{project_name}.apwz")
Aspen_path = f"C:\\Users\\user\\Documents\\Carbonfix_package\\{project_name}.apwz"
# Aspen_path = "C:\\Users\\user\\Documents\\Carbonfix_package\\Simulation-DEC.apwz"
Aspen, Asp = aspen_tools.connect_aspen(Aspen_path)


## Input Component CAS Numbers and Set Up Aspen

Enter the **CAS numbers** for all components (feed and product)


In [ ]:
Feed_CAS_list = ["1333-74-0","124-38-9"]
Product_CAS_list = ["67-56-1" ,"7732-18-5"]

aspen_tools.input_CAS_COMP_list(Aspen,Asp, Feed_CAS_list+Product_CAS_list) #commit this line if aspen file has already been created


Component  | CAS Number   | Component ID | Status
1          | 1333-74-0    | COMP1      | Match
2          | 124-38-9     | COMP2      | Match
3          | 67-56-1      | COMP3      | Match
4          | 7732-18-5    | COMP4      | Match


## Setting Temperature and Pressure Range and Feed composition



In [ ]:
P_list = [1,5,10,25,50] #atm
T_lower = 298 #K
T_upper = 488 #K
increment = 10 #K
T_list = [i for i in range(T_lower, T_upper+1, increment)]
f = 0.5 # irreversibility factor
n_in = [3,1,0,0]  #mol of components in Feed_CAS_list]
Recycle = True 
NCOMP = len(Feed_CAS_list + Product_CAS_list)

## Calculate Conversion

In [4]:
n_out = analysis.Xeq(Aspen, Asp, T_list, P_list, n_in)
Xeq = analysis.compute_Xeq_df(n_out,n_in,T_list,P_list,comp_index=1) # comp index for conversion calculation
Xeq

,1,5,10,25,50
298,0.392809,0.944550,0.981230,0.996485,0.999971
308,0.289686,0.890300,0.961998,0.991596,0.998511
318,0.236233,0.791084,0.928354,0.982802,0.995446
328,0.191444,0.621095,0.870488,0.968128,0.990021
338,0.154668,0.435841,0.771774,0.944466,0.981294
348,0.124900,0.381960,0.617127,0.906791,0.967826
358,0.101024,0.332196,0.473321,0.847022,0.947382
368,0.081971,0.287152,0.424004,0.753501,0.916567
378,0.066795,0.247084,0.377382,0.621488,0.870225
388,0.054701,0.211951,0.334053,0.521045,0.800885


## Calculate Boiling point

In [5]:
Tb_list = analysis.calc_Tb(Aspen, Asp, T_list, P_list,NCOMP)

Pressure      | COMP1             | COMP2             | COMP3             | COMP4             
1 atm         | < 298 K            K| < 298 K            K| 337.9              K| 374.0              K
5 atm         | < 298 K            K| < 298 K            K| 385.4              K| 426.25             K
10 atm        | < 298 K            K| < 298 K            K| 411.05             K| 453.8              K
25 atm        | < 298 K            K| < 298 K            K| 450.0              K| > 488 K            K
50 atm        | < 298 K            K| < 298 K            K| 486.1              K| > 488 K            K


## Calculate Process Work

In [ ]:
Aspen, Asp, W_total, W_rev, W_HC_sgen, W_M_sgen, W_R_sgen, W_S_sgen, W_CD_sgen, CF, CF_delta = analysis.calc_work(Aspen, Asp, Aspen_path,T_list, P_list, Tb_list, n_in, n_out,f,NCOMP,recycle=Recycle)

COMP 1 can be fully recycled
Running n_in_Bp...
Running n_inr_Bp...
Running n_inr_Bm...
Running n_outr_Bm...
Running n_outr_Bp...
Running n_out_Bp...
Running n_out_Bp0...


## Export to Excel

In [ ]:
file_name = f"{project_name}.xlsx"
analysis.export_to_excel(file_name, T_list, P_list, Tb_list, Xeq, W_total, W_rev, W_HC_sgen, W_M_sgen, W_R_sgen, W_S_sgen, W_CD_sgen, CF, CF_delta)

In [8]:
aspen_tools.close_aspen(Aspen)